<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [20]:
import random
import datetime
import numpy as np
import pandas as pd


class G:
    ITERATIONS = 100
    DURATIONS = 6120

G.target_times = {
    1: 2,
    2: 3,
    3: 5
}

G.specialties = {
    1: '1',
    2: '2',
    3: '3',
    4: '4',
    5: '5'
}

def update_globals(urg_time, non_time):
    G.process_times = {
                        1: urg_time,
                        2: .3*(non_time - urg_time),
                        3: non_time
                    }
    
    

def create_arrival_times(sim_time, arr_rate):
    arrival_times = []
    time = 0
    while time < sim_time:
        time += np.random.exponential(arr_rate)
        arrival_times.append(time)
    return arrival_times

def create_medical_images(arrival_times):
    med_images = []
    for i, t in enumerate(arrival_times):
        med_images.append(MedicalImage(i, t, random.sample(list(G.target_times.keys()), 1)[0], random.sample(list(G.specialties.keys()), 1)[0]))
    return med_images

def create_radiologists(num_rads):
    radiologists = []
    for i in range(5):
        specialties_temp = random.sample(list(G.specialties.keys()), random.randrange(2,num_rads))
        radiologists.append(Radiologist(i, specialties_temp))
    return radiologists



def create_initial_events(med_images):
    events=[]
    for img in med_images:
        events.append([img.time_created, 'New Job', img])
    return events


def start_simulation(events, med_images, radiologists):
    s = SystemState(events, med_images, radiologists)
    s.run_simulation()


class MedicalImage(object):    
    def __init__(self, img_id, time_created, urgency, image_type):#, modality, speciality, urgency, image_label):
        self.img_id = img_id
        self.time_created = time_created
        self.urgency = urgency
        self.image_type = image_type
        self.target_time = G.target_times[urgency]
        self.time_remaining = G.target_times[urgency]
        self.est_process_time = G.process_times[urgency]
        self.in_queues = []   #keep track on which queues image is in [rad_id, position]
        self.time_seen = 0
        self.time_done = 0
        self.rad_seen = "None"
        
    def update_time_remaining(self, t):
        self.time_remaining = self.target_time - (t - self.time_created)
        
        
class Radiologist:
    def __init__(self, rad_id, specialties, working=True):
        self.queue = []
        self.queue_data = []#[med_image, image_id, image_urgency, time_left, est_time]
        self.rad_id = rad_id
        self.specialties = specialties
        self.is_working = working
        self.images_served = []
        self.idle_times = []
        self.time_of_last_idle = 0
        self.time_last_not_idle = 0
        self.busy_times = []
        self.time = 0
        self.time_current_job_start = 0
        self.time_of_step = 0
        self.queue_length = []
        self.service_starts = []
        self.service_ends = []
        self.service_time = []  
        
    def get_stats(self):
        return self.idle_times, self.busy_times, self.queue_length, self.service_starts, self.service_ends, self.service_time 
        
    def show_queue(self):
        return self.queue
    
    def add_job(self, med_image, time):
        self.queue.append(med_image)    #each customer is represented by the time it will take for them to be served
        self.queue_data.append([med_image, med_image.img_id, med_image.urgency, med_image.time_remaining, med_image.est_process_time, med_image.est_process_time]) #[image_id, image_urgency, time_left, est_time]
        #print(self.show_queue())
        
    def update_queue(self, time):
        for img in self.queue:
            img.update_time_remaining(time)
        #sort_queue()        
    #def sort_queue(self):
                   
        
        
class SystemState:
    def __init__(self, events, images, rads):
        self.time = 0
        self.events = events
        self.images = images
        self.rads = rads
        self.rads_working = rads
        self.rads_not_working = []
        self.events_history = []
        self.queue_lengths = []
        self.time_steps = []
        self.img_table = pd.DataFrame(columns=['img_id','urgency', 'create_time','seen_time', 'finished', 'time_w_rad', 'total_time'])
        self.rad_table = pd.DataFrame()
        
    def create_event(self, time, event_type, obj):
        self.events.append([time, event_type, obj])
        self.events = sorted(self.events, key=lambda x: x[0])

    def update_img_table(self, med_img):
        column_names = ['img_id','urgency', 'create_time','seen_time', 'finished', 'time_w_rad', 'total_time']
        values = [med_img.img_id, med_img.urgency, med_img.time_created, med_img.time_seen, self.time, self.time - med_img.time_seen, self.time - med_img.time_created] #[[med_img.time_created], [med_img.time_seen], [self.time], [self.time - med_img.time_seen], [self.time - med_img.time_created]]
        temp_df = pd.DataFrame(values).T
        temp_df.columns = column_names
        self.img_table = self.img_table.append(temp_df, ignore_index = True)
        
    def process_event(self):
        event = self.events[0]
        self.events_history.append(event)
        self.time = event[0]       
        event_type = event[1]
        del self.events[0]
        temp_list = []
        for r in self.rads:
            temp_list.append(len(r.queue))
        self.queue_lengths.append(temp_list)
        self.time_steps.append(self.time)
            
        if event_type == "New Job":
            self.distribute_job(event[2])
        elif event_type == "Job Done":
            rad = event[2]
            self.complete_job(rad)
        print("Event processed")
        if len(self.events) > 0:
            self.process_event()
        else:
            print("Simulation complete")
                
    def distribute_job(self, med_image):
        image_type = med_image.image_type
        capable_rads = []
        for rad in self.rads_working:      #finds radiologists capable of working on image
            if image_type in rad.specialties:
                capable_rads.append(rad)
        for rad in capable_rads:
            rad.add_job(med_image, self.time)
            med_image.in_queues.append(rad)    #keep track of which rads have image in queue
            if len(rad.queue)==1:
                self.start_job(rad)
                break         
        self.update_queues()
             
    def update_queues(self):
        for rad in self.rads_working:
            rad.update_queue(self.time)
                
    def start_job(self, rad):
        med_image = rad.queue[0]
        image_type = med_image.image_type
        urgency = med_image.urgency
        rad.service_starts = self.time
        med_image.time_seen = self.time
        med_image.rad_seen = rad.rad_id
        self.events_history.append([self.time, "Job Started", med_image])
        process_time = np.random.exponential(G.target_times[urgency])
        self.create_event(self.time+process_time, "Job Done", rad)
        print(f"Image {med_image.img_id} is seen by radiologist {rad.rad_id} at {self.time}")
        for r in med_image.in_queues:
            if r != rad:
                r.queue.remove(med_image)           
        
    def complete_job(self, rad):
        med_image = rad.queue[0]
        self.update_img_table(med_image)
        rad.images_served.append(med_image.img_id)
        rad.service_ends.append(self.time)
        med_image.time_done = self.time
        print(f"Image {med_image.img_id} is done by radiologist {rad.rad_id} at {self.time}")
        del rad.queue[0]
        if len(rad.queue) > 0:
            self.start_job(rad)

    def run_simulation(self):
        self.process_event()

In [21]:
def gen_system_state(sim_time, rads_count, arr_rate, urg_time, non_time):
    #Define urgency times
    update_globals(urg_time, non_time)
    #Create the intervals
    arrival_times = create_arrival_times(sim_time, arr_rate)
    #Create the images with their arrival time_seen
    med_images = create_medical_images(arrival_times)
    #Create the radiologists
    radiologists = create_radiologists(rads_count)
    #Create the image arrival events
    events = create_initial_events(med_images)
    s = SystemState(events, med_images, radiologists)
    return s

def sim(rads_count, arr_rate, urg_time, non_time):
    sim_time = 60        
    s = gen_system_state(sim_time, rads_count, arr_rate, urg_time, non_time)
    s.run_simulation()    
    return s


def plot_queue_lengths(s):
    fig, ax = plt.subplots()
    for i in range(len(s.queue_lengths[0])):
        plt.plot(s.time_steps, [item[i] for item in s.queue_lengths])
        
        
def plt_mean_queue_length(s_list):
    fig, ax = plt.subplots()
    for s in s_list:
        plt.plot(s.time_steps, pd.DataFrame(s.queue_lengths).sum(axis=1), label=f"{len(s.rads)}")
    plt.xlabel("time")
    plt.ylabel("Mean Queue Length")
    plt.legend()
    plt.show()

In [22]:
rads_count = 3
arr_rate = 1
urg_time = 2
non_time = 5

s1 = sim(rads_count, arr_rate, urg_time, non_time)

Image 0 is seen by radiologist 0 at 2.3033754218451326
Event processed
Image 1 is seen by radiologist 2 at 2.799788227311942
Event processed
Image 1 is done by radiologist 2 at 3.1143385739643716
Event processed
Image 2 is seen by radiologist 1 at 4.791440231011986
Event processed
Image 2 is done by radiologist 1 at 4.9086646707334545
Event processed
Image 0 is done by radiologist 0 at 4.955112885812923
Event processed
Image 3 is seen by radiologist 3 at 5.409556317352189
Event processed
Event processed
Image 5 is seen by radiologist 4 at 6.224410019395144
Event processed
Image 5 is done by radiologist 4 at 6.83750594503357
Event processed
Image 6 is seen by radiologist 0 at 6.880069058409217
Event processed
Image 6 is done by radiologist 0 at 7.473072096439914
Event processed
Image 3 is done by radiologist 3 at 8.26166078832309
Event processed
Image 7 is seen by radiologist 0 at 8.575970507934125
Event processed
Image 8 is seen by radiologist 1 at 9.501339921223014
Event processed
Ima

In [23]:
s1.img_table

,img_id,urgency,create_time,seen_time,finished,time_w_rad,total_time
0,1.0,2.0,2.799788,2.799788,3.114339,0.314550,0.314550
1,2.0,1.0,4.791440,4.791440,4.908665,0.117224,0.117224
2,0.0,1.0,2.303375,2.303375,4.955113,2.651737,2.651737
3,5.0,3.0,6.224410,6.224410,6.837506,0.613096,0.613096
4,6.0,2.0,6.880069,6.880069,7.473072,0.593003,0.593003
5,3.0,3.0,5.409556,5.409556,8.261661,2.852104,2.852104
6,8.0,1.0,9.501340,9.501340,10.677786,1.176446,1.176446
7,10.0,1.0,14.155529,14.155529,14.189059,0.033530,0.033530
8,7.0,2.0,8.575971,8.575971,16.049309,7.473339,7.473339
9,9.0,3.0,11.974053,11.974053,16.865580,4.891528,4.891528


In [ ]:
def rad_table(agent_count, call_table_df):
    '''
    Input 1: 'agent_count': number of agents staffed
    Input 2: 'call_table_df': detailed call table with call start times and AHTs
    '''
    agent_status = [1 for x in range(agent_count)]
    col_name = ['rad_id','img_id', 'img_start_time', 'img_end_time','time_to_compl', 'call_wait_time', 'call_handle_time_elapsed',
                'intvl_start_time','call_arrival_time', 'intvl_time_elapsed', 'call_end_time_elapsed', 'call_wait_time_elapsed']
    agent_table_df = pd.DataFrame(columns=col_name)   
    
    for i in range(len(call_table_df)):
        
        int_df = pd.DataFrame({col_name[0]: i,
                               col_name[1]: agentPicked,
                               col_name[2]: call_handle_start_time,
                               col_name[3]: call_table_df['call_aht'][i],
                               col_name[4]: timeAddition(call_handle_start_time, [0,0,call_table_df['call_aht'][i]]),
                               col_name[5]: call_wait_time,
                               col_name[6]: timeElapsed(call_handle_start_time),
                               col_name[7]: call_table_df['intvl_start_time'][i],
                               col_name[8]: call_table_df['call_start_time'][i],
                               col_name[9]: call_table_df['intvl_time_elapsed'][i],
                               col_name[10]: timeElapsed(timeAddition(call_handle_start_time, [0,0,call_table_df['call_aht'][i]])),
                               col_name[11]: timeElapsed(call_wait_time)
                              }, index=[0])
        #print(int_df)
        agent_table_df = agent_table_df.append(int_df, ignore_index=True)
        
    return rad_table_df
